#### Author: Faolán Hamilton

Get the data from this link - https://cli.fusio.net/cli/climate_data/webdata/hly4935.csv

## Part 1 - 60%
Plot:

- The temperature
- The mean temperature each day
- The mean temperature for each month


## Part 2 - 40%

Plot:

- The Windspeed (there is data missing from this column)
- The rolling windspeed (say over 24 hours)
- The max windspeed for each day
- The monthly mean of the daily max windspeeds (yer I am being nasty here)

You do not need to over comment your code. Marks will be given for how nice the plots are.

In [45]:
from datetime import datetime as dt

from datetime import date as date

import pandas as pd

import numpy as np

##### To get the csv to open, I thought I had to download the data from the link, bring it into the assignment folder and delete the first several rows to clean the data - all I had to do was use the 'skiprows' function!

In [46]:
# read in data
df = pd.read_csv("https://cli.fusio.net/cli/climate_data/webdata/hly4935.csv", skiprows=23)
df.head(2)

C:\Users\ClaireReilly\AppData\Local\Temp\ipykernel_32628\4014489085.py:2: DtypeWarning: Columns (2,10,12,14,15,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("https://cli.fusio.net/cli/climate_data/webdata/hly4935.csv", skiprows=23)


,date,ind,rain,ind.1,temp,ind.2,wetb,dewpt,vappr,rhum,...,ind.3,wdsp,ind.4,wddir,ww,w,sun,vis,clht,clamt
0,10-apr-1996 14:00,0,0.0,0,11.5,0,8.1,3.9,0.0,0,...,0,0,0,0,25,81,0.0,35000,32,5
1,31-jul-1996 08:00,0,0.0,0,11.5,0,11.1,10.7,0.0,0,...,0,0,0,0,25,82,0.0,40000,45,5


#### Looking at the question, I will only need temperature and windspeed so it is a good idea to remove unneccessary columns

In [47]:
# see all column names
headers = df.columns.tolist()
headers

['date',
 'ind',
 'rain',
 'ind.1',
 'temp',
 'ind.2',
 'wetb',
 'dewpt',
 'vappr',
 'rhum',
 'msl',
 'ind.3',
 'wdsp',
 'ind.4',
 'wddir',
 'ww',
 'w',
 'sun',
 'vis',
 'clht',
 'clamt']

In [48]:
# remove the headers that are not relevant
drop_col_list = ['ind','rain','ind.1', 'ind.2', 'wetb', 'dewpt', 'vappr', 'rhum', 'msl', 'ind.3','ind.4', 'wddir', 'ww', 'w','sun','vis','clht','clamt']
df.drop(columns=drop_col_list, inplace=True)
df.head(2)

,date,temp,wdsp
0,10-apr-1996 14:00,11.5,0
1,31-jul-1996 08:00,11.5,0


In [49]:
# I want to see the dtype of each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256434 entries, 0 to 256433
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   date    256434 non-null  object 
 1   temp    256434 non-null  float64
 2   wdsp    256434 non-null  object 
dtypes: float64(1), object(2)
memory usage: 5.9+ MB


### I want to clean up the dtype, starting with the date column

##### Datetime conversion source: https://www.geeksforgeeks.org/pandas/convert-the-column-type-from-string-to-datetime-format-in-pandas-dataframe/

In [50]:
# convert the date column to a datetime format
df['date'] = pd.to_datetime(df['date'], format="%d-%b-%Y %H:%M")
df['date']

0        1996-04-10 14:00:00
1        1996-07-31 08:00:00
2        1996-07-31 09:00:00
3        1996-07-31 10:00:00
4        1996-07-31 11:00:00
                 ...        
256429   2025-10-31 20:00:00
256430   2025-10-31 21:00:00
256431   2025-10-31 22:00:00
256432   2025-10-31 23:00:00
256433   2025-11-01 00:00:00
Name: date, Length: 256434, dtype: datetime64[ns]

In [51]:
# set the date column as the index to search by dates easily
df.set_index('date', inplace=True)
df.head(2)

,temp,wdsp
date,,
1996-04-10 14:00:00,11.5,0
1996-07-31 08:00:00,11.5,0


In [52]:
# set the Wind speed column to numeric value
df['wdsp'] = pd.to_numeric(df['wdsp'], errors='coerce')
df.head(2)

,temp,wdsp
date,,
1996-04-10 14:00:00,11.5,0.0
1996-07-31 08:00:00,11.5,0.0


### The heading names are not entirely clear, I want to rename them

###### renaming columns (https://www.geeksforgeeks.org/python/how-to-rename-multiple-column-headers-in-a-pandas-dataframe/)

###### renaming index (https://stackoverflow.com/questions/19851005/rename-pandas-dataframe-index)

In [53]:
#improve the naming convention of the headings

df.columns = df.columns.str.replace('temp', 'Temperature (C)').str.replace('wdsp', 'Wind Speed (km)')
df.index.names = ['Date and Time']
df.head(2)

,Temperature (C),Wind Speed (km)
Date and Time,,
1996-04-10 14:00:00,11.5,0.0
1996-07-31 08:00:00,11.5,0.0


##### I want to check if there are any null values

In [54]:
# check for nulls
df.isnull().sum()

Temperature (C)     0
Wind Speed (km)    50
dtype: int64

##### check which rows nulls are in source (https://stackoverflow.com/questions/27159189/find-empty-or-nan-entry-in-pandas-dataframe)

In [55]:
# See which rows have empty values to double check later
df[(df['Wind Speed (km)'].isnull())].index

DatetimeIndex(['1996-08-01 22:00:00', '1996-09-27 13:00:00',
               '1996-10-14 02:00:00', '1996-10-14 03:00:00',
               '1996-10-14 04:00:00', '1996-10-14 05:00:00',
               '1996-10-14 06:00:00', '1996-10-14 07:00:00',
               '1996-10-14 08:00:00', '1996-10-14 09:00:00',
               '1996-10-15 01:00:00', '1996-10-15 02:00:00',
               '1996-10-15 03:00:00', '1996-10-15 04:00:00',
               '1996-10-15 05:00:00', '1996-10-15 06:00:00',
               '1996-10-15 07:00:00', '1996-10-15 08:00:00',
               '1996-10-15 17:00:00', '1996-10-15 18:00:00',
               '1996-10-15 19:00:00', '1996-10-15 20:00:00',
               '1996-10-15 21:00:00', '1996-10-15 22:00:00',
               '1996-10-15 23:00:00', '1996-10-16 00:00:00',
               '1996-10-16 01:00:00', '1996-10-16 02:00:00',
               '1996-10-16 03:00:00', '1996-10-16 04:00:00',
               '1996-10-16 05:00:00', '1996-10-16 06:00:00',
               '1996-10-

In [61]:
# check what the NaN value looks like
df.loc['1996-08-01 22:00:00']

Temperature (C)    10.8
Wind Speed (km)     0.0
Name: 1996-08-01 22:00:00, dtype: object

#### There are 50 null values in Wind Speed, I will need to clean this data up - my chosen method will be to replace the null values with 0 to be consistent with the existing data structure

In [62]:
# fill NaN value with '0.0'
df.fillna(value='0.0', inplace=True)
df.head(2)

,Temperature (C),Wind Speed (km)
Date and Time,,
1996-04-10 14:00:00,11.5,0.0
1996-07-31 08:00:00,11.5,0.0


In [63]:
# check to see if that worked
df.loc['1996-08-01 22:00:00']

Temperature (C)    10.8
Wind Speed (km)     0.0
Name: 1996-08-01 22:00:00, dtype: object

------------------------------------------------------------------------

#### https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.interpolate.html

In [59]:
#df['temp'].interpolate()

### numpy documentation - mean (https://numpy.org/doc/stable/reference/generated/numpy.mean.html)

In [60]:
# mean temperature for all datetimes 
#tempmean = np.mean(df['temp'])
#tempmean